In [ ]:
'''
Find the price correlated HK stocks for Pair strategy 
Buy the stock when it is relatively under-priced to the correlated stock 
and sell the corresponding over-priced stock
As they're price correlated (statistically), assume they will go back to the similar price ratio later
Hopefully, the under-priced stock will goes up later while the over-priced one will drop
'''

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

from datetime import datetime

import numpy as np
import pandas as pd
import pandas_datareader.data as web

# webscraping library
import requests
from bs4 import BeautifulSoup

# Download the font to display Chinese
!wget https://github.com/kenwkliu/ideas/raw/master/colab/data/simhei.ttf
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
CNFont = FontProperties(fname='/content/simhei.ttf')

# Google colab interactive table
%load_ext google.colab.data_table 
%matplotlib inline

In [ ]:
# Get the stock information from Yahoo
stockInfo = web.get_quote_yahoo('0700.HK')
stockInfo

In [ ]:
# show the column names
stockInfo.columns

In [ ]:
# Get the pre-downloaded all HK stocks information as of 20200519
stocksInfo = pd.read_csv('https://raw.githubusercontent.com/kenwkliu/ideas/master/colab/data/20200519-HK-Stocks.csv')
stocksInfo

In [ ]:
# Look at one stock
CODE = '0005.HK'
stocksInfo[(stocksInfo.code == CODE)]

In [ ]:
# add a new colume 'turnover' 
stocksInfo['turnover'] = round(stocksInfo['regularMarketPreviousClose'] * stocksInfo['averageDailyVolume10Day'])

# Look at the subset of useful columns and sort by turnover
SELECTED_COLUMNS = ['code', 'quoteType', 'shortName', 'longName', 'marketCap', 'turnover', 'regularMarketPreviousClose', 'averageDailyVolume10Day']
SORT_BY = 'turnover'
IS_ASCENDING = False

stocksSubInfo = stocksInfo[SELECTED_COLUMNS].sort_values(by=[SORT_BY], ascending=IS_ASCENDING)
stocksSubInfo

In [ ]:
# We're only interested in trading liquid equity names
# Hence select only EQUITY and turnover is bigger than certain numbers
QUOTE_TYPE = 'EQUITY'
MIN_TURNOVER = 100000000  # 100 millions
SELECTED_COLUMNS = ['code', 'shortName', 'turnover']

stocksFilteredInfo = stocksSubInfo[(stocksSubInfo.quoteType == QUOTE_TYPE) & (stocksSubInfo.turnover > MIN_TURNOVER)][SELECTED_COLUMNS]
stocksFilteredInfo

In [ ]:
# Use Webscraping to extract HK stock Chinese names from wiki web site
hk_url = 'https://zh-yue.wikipedia.org/wiki/%E9%A6%99%E6%B8%AF%E4%B8%8A%E5%B8%82%E5%85%AC%E5%8F%B8%E4%B8%80%E8%A6%BD'
html = requests.get(hk_url)
soup = BeautifulSoup(html.text, 'html.parser')
soup

In [ ]:
# Read the tags line by line and scrape the stock code and names
code = []
name = []
STOCK_SUFFIX = '.HK'

a_tags = soup.find("div", attrs={"id":"mw-content-text"})
all_li = a_tags.find_all("li", attrs={"class":""})

for li in all_li:
    content = li.text.strip()
    code.append(str(content[:4] + STOCK_SUFFIX))
    name.append(content[4:].strip())
    
chiNames = pd.DataFrame(index=code, data=name)
chiNames = chiNames.reset_index()
chiNames.columns = ["code", "chiName"]
chiNames

In [ ]:
# Merge the Chinese names (similar to Excel vlookup)
results = stocksFilteredInfo.merge(chiNames, on='code', how='left')
results[['code', 'shortName', 'chiName', 'turnover']] 

In [ ]:
# download the latest stock price from Yahoo
SOURCE = 'yahoo'
start = '2020' # accepts strings
end = datetime.today()

columnNames = []
stocksDownload = []

# use results.shortName for display company names in English 
# use results.chiName for display company names in Chinese 
for code, name in zip(results.code, results.chiName):
  try:
    stocksDownload.append(web.DataReader(code, SOURCE, start=start, end=end))
    columnNames.append(name)
    print("Completed:", code, name)
  except Exception as e:
    print(code, name, ": error:", e)

print("All Completed")    

In [ ]:
# look at the downloaded stock price
index = 0
print(columnNames[index])
stocksDownload[index]

In [ ]:
# Only use "Adj Close"
adjCloses = [stock['Adj Close'] for stock in stocksDownload]

# Combine the stocks adjusted closes to one pandas data frame
stocks = pd.concat(adjCloses, axis=1)
stocks.columns = columnNames
stocks

In [ ]:
# Export the data to csv file for downloading
stocks.to_csv("hkStocksAdjClosePx.csv", encoding='utf_8_sig')

In [ ]:
# Run stocks correlation
stocksCorr = stocks.corr()
stocksCorr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
# Filter the correlated stocks pair with the THRESHOLD
THRESHOLD = 0.98
highCorr = np.where(((stocksCorr >= THRESHOLD) & (stocksCorr < 1)))
print("Number of pairs found:", int(len(highCorr[0])/2))


# Print the Paired stocks
count = 0
pairs = {}
for i in range(len(highCorr[0])):
  a, b = highCorr[0][i], highCorr[1][i]
  revKey = str(b) + str(a)

  # prevent duplicated plot as (a,b) pair is the same as (b,a) pair
  if revKey not in pairs:
    key = str(a) +str(b)
    pairs[key] = [a, b]
    print(count, ":", columnNames[a], "<->", columnNames[b])
    count += 1

In [ ]:
# Plot the highly correlated stock pairs 

for k, pair in pairs.items():
  stocks[[columnNames[pair[0]], columnNames[pair[1]]]].plot()
  plt.legend(prop = CNFont)

In [ ]:
# Choose a stock pair
INDEX = 0
stock_pair = list(pairs.values())[INDEX]

PAIR_STOCK_A = columnNames[stock_pair[0]]
PAIR_STOCK_B = columnNames[stock_pair[1]]

# Calculate the stock price ratios
stocks['A/B'] = stocks[PAIR_STOCK_A] / stocks[PAIR_STOCK_B]
stocks['B/A'] = stocks[PAIR_STOCK_B] / stocks[PAIR_STOCK_A]

stocks[[PAIR_STOCK_A, PAIR_STOCK_B, 'A/B', 'B/A']]

In [ ]:
# Calculate the average price ratio of the Pair 
# Trade if the current price ratio is significantly different 
PX_RATIO_THRESHOLD = 0.05

# Calculate the average Buy PAIR_STOCK_A / Sell PAIR_STOCK_B ratio
avgPxRatio = stocks['A/B'].mean()
print("Average BUY/SELL price ratio of", PAIR_STOCK_A, "/", PAIR_STOCK_B, ":", avgPxRatio)

execPxRatio = avgPxRatio * (1 - PX_RATIO_THRESHOLD)
print("Buy", PAIR_STOCK_A, "and Sell", PAIR_STOCK_B, "when the BUY/SELL price ratio is smaller than:", execPxRatio)

print(f'-'*100)
print("The ratio can be reached when relatively ... ")
print(PAIR_STOCK_A, "goes down and become under-valued and should buy it")
print(PAIR_STOCK_B, "goes up and become over-valued and should sell it")

In [ ]:
# Calculate the average Buy PAIR_STOCK_B / Sell PAIR_STOCK_A ratio
avgPxRatio = stocks['B/A'].mean()
print("Average BUY/SELL price ratio of", PAIR_STOCK_B, "/", PAIR_STOCK_A, ":", avgPxRatio)

execPxRatio = avgPxRatio * (1 - PX_RATIO_THRESHOLD)
print("Buy", PAIR_STOCK_B, "and Sell", PAIR_STOCK_A, "when the BUY/SELL price ratio is smaller than:", execPxRatio)

print(f'-'*100)
print("The ratio can be reached when relatively ... ")
print(PAIR_STOCK_B, "goes down and become under-valued and should buy it")
print(PAIR_STOCK_A, "goes up and become over-valued and should sell it")

In [ ]:
''' Suggestions
- Find dataset with stock sector informatin and filter the stock pairs with same sector only
- Or run a clustering alogrithms (such as k-mean or DBScan) to group similar stocks together and trade Pairs only within the same cluster
'''